In [1]:
import json
import pandas as pd
import numpy as np

## Config

In [ ]:
#測試資料檔案位置
inputData = "./2022-test-v1.xlsx"

#標準化模型儲存資料夾位置
stdModelFile = "./model/"

#推論模型儲存資料夾位置
pklFile = "./model/"

#預測檔案輸出位置
outputFile = pklFile+"111924_TestResult.xlsx"

#模型blending權重檔案
blendingWeightFile = pklFile+"blending_weight.txt"

## load test data

In [10]:
testData = pd.read_excel(inputData)

In [13]:
testData.shape

(100, 61)

## data process

In [12]:
#刪除訓練時沒用到欄位
totalDropCol =['clean_ph5', 'clean_ph7', 'clean_ph4','clean_pressure11', 'clean_pressure21', 'clean_pressure23', 'clean_pressure72', 'clean_pressure101', 'painting_g6_act_a_air', 'painting_g6_act_f_air', 'painting_g6_act_t_air', 'painting_g6_act_hvv', 'painting_g6_act_hvc', 'painting_g12_act_a_air', 'painting_g12_act_f_air', 'painting_g12_act_t_air', 'painting_g12_act_hvv', 'painting_g12_act_hvc', 'env_rpi09_hum', 'env_rpi09_lux', 'env_rpi09_temp','clean_pressure92', 'clean_pressure71', 'oven_pb2', 'oven_a2', 'oven_b2', 'oven_b3', 'painting_g1_act_f_air', 'painting_g1_act_t_air', 'painting_g1_act_hvv', 'painting_g1_act_hvc', 'painting_g7_act_a_air', 'painting_g7_act_f_air', 'painting_g7_act_t_air', 'painting_g7_act_hvv', 'painting_g2_act_t_air', 'painting_g8_act_a_air', 'painting_g8_act_t_air', 'painting_g3_act_t_air', 'painting_g9_act_t_air', 'painting_g9_act_f_air', 'painting_g4_act_f_air', 'painting_g4_act_t_air', 'painting_g4_act_hvv', 'painting_g10_act_a_air', 'painting_g10_act_f_air', 'painting_g10_act_t_air', 'painting_g10_act_hvv', 'painting_g5_act_f_air', 'painting_g5_act_t_air', 'painting_g5_act_hvv', 'painting_g11_act_f_air', 'painting_g11_act_t_air', 'painting_g11_act_hvv', 'env_rpi05_pm10', 'env_rpi05_pm25', 'env_rpi07_pm10', 'env_rpi07_pm25', 'env_rpi09_pm10', 'env_rpi09_pm25', 'env_rpi14_pm10', 'env_rpi14_pm25', 'env_rpi15_pm10', 'env_rpi15_pm25']

testData = testData.drop(totalDropCol, axis=1)

## 標準化

In [17]:
from sklearn import preprocessing
import pickle

with open(stdModelFile+"stdMode.pkl","rb") as file:
    std = pickle.load(file)

X = std.transform(testData)

## inference

In [29]:
# load blending model weight
filename = blendingWeightFile
with open(filename, "r") as file:
    weights = json.load(file)

In [41]:
#inference blending
result = {}
pointArr = ["5","6","7","8","9","10"]

for i in pointArr:
    pointStr = "point{}".format(i)
    tempdf = pd.read_json(weights[pointStr],orient="records")
    
    pred = 0
    for i, item in tempdf.iterrows():

        filename = pklFile+pointStr+"/"+item["model"]+".pkl"
        with open(filename,"rb") as file:
            model = pickle.load(file)
            pred += float(item["optimised_wts"])*model.predict(X)
        
    result["sensor_{}_i_value".format(pointStr)]=pred

In [50]:
#結果輸出
tempdf = pd.DataFrame.from_dict(result)
tempdf.index = tempdf.index+1
tempdf.to_excel(outputFile)